In [1]:
import pandas as pd
import numpy as np
from numpy import random

# random.seed(100)

* TicketID
* Datetime
* Type : ["Security", "Account", "Database", "Application", "Network"]
* Priority
* RequestingDepartment : ["Sales", "Operators", "Production", "Reporting", "Receiving", "Warehouse", "Marketing"]

In [18]:
random.randint(0, 2, 10)

array([1, 1, 0, 0, 1, 0, 0, 0, 1, 1], dtype=int32)

In [39]:
import pandas as pd
import numpy as np

# Extract a single SubmitTime (e.g., from iter_df)
SubmitTime = iter_df["SubmitTime"].iloc[0]  # or loop logic, if needed

# Assign to all rows
join_df["SubmitTime"] = pd.to_datetime(SubmitTime)  # Single timestamp, broadcasted
join_df["DurationMinutes"] = pd.to_timedelta(np.random.randint(0, 121, size=cnt_tickets), unit='m')
join_df["CloseTime"] = join_df["SubmitTime"] + join_df["DurationMinutes"]   

join_df

,TicketID,SubmitTime,DurationMinutes,CloseTime,Type,Priority,Department
0,67698,2025-01-01,0 days 00:14:00,2025-01-01 00:14:00,Account,0,Receiving
1,47523,2025-01-01,0 days 00:34:00,2025-01-01 00:34:00,Network,0,Marketing
2,72751,2025-01-01,0 days 00:11:00,2025-01-01 00:11:00,Security,1,Receiving


In [41]:
date_df = pd.DataFrame({"Date": pd.date_range(start="2025-01-01", end="2025-08-31").tolist()})
hours_df = pd.DataFrame({"Hours": np.arange(0, 24, 1)})

date_df = date_df.join(hours_df, how="cross")

iter_df = pd.DataFrame({"SubmitTime": date_df["Date"] + pd.to_timedelta(date_df["Hours"], unit="h")})
df = pd.DataFrame(columns=["TicketID", "SubmitTime", "CloseTime", "DurationMinutes", "Type", "Priority", "Department"])

types = ["Security", "Account", "Database", "Application", "Network"]
departments = ["Sales", "Operatorations", "Production", "Reporting", "Receiving", "Warehouse", "Marketing"]

for i in range(len(iter_df)):
    SubmitTime = iter_df["SubmitTime"].iloc[i]
    cnt_tickets = random.randint(0, 5) # Tickets per hour

    join_df = pd.DataFrame()

    join_df["TicketID"] = random.randint(10000, 99999, size=cnt_tickets)
    join_df["SubmitTime"] = pd.to_datetime(SubmitTime)  # Single timestamp, broadcasted
    join_df["DurationMinutes"] = pd.to_timedelta(np.random.randint(0, 121, size=cnt_tickets), unit='m')
    join_df["CloseTime"] = join_df["SubmitTime"] + join_df["DurationMinutes"]   
    join_df["Type"] = random.choice(types, size=cnt_tickets)
    join_df["Priority"] = random.randint(0, 2, cnt_tickets)
    join_df["Department"] = random.choice(departments, size=cnt_tickets)

    join_df["DurationMinutes"] = (join_df["DurationMinutes"] / pd.Timedelta(minutes=1)).astype(int)   

    df = pd.concat([df, join_df])

df

,TicketID,SubmitTime,CloseTime,DurationMinutes,Type,Priority,Department
0,39075,2025-01-01 00:00:00,2025-01-01 00:36:00,36,Account,1,Operatorations
1,73426,2025-01-01 00:00:00,2025-01-01 00:54:00,54,Database,1,Production
2,98901,2025-01-01 00:00:00,2025-01-01 00:05:00,5,Database,1,Receiving
3,66805,2025-01-01 00:00:00,2025-01-01 01:45:00,105,Account,0,Reporting
0,98164,2025-01-01 01:00:00,2025-01-01 02:53:00,113,Network,0,Reporting
...,...,...,...,...,...,...,...
2,38565,2025-08-31 21:00:00,2025-08-31 22:27:00,87,Application,0,Sales
0,96977,2025-08-31 22:00:00,2025-08-31 22:32:00,32,Network,1,Sales
1,84387,2025-08-31 22:00:00,2025-08-31 22:05:00,5,Account,1,Sales
0,94792,2025-08-31 23:00:00,2025-09-01 00:27:00,87,Security,1,Production


In [ ]:
pd.DataFrame({"Department": departments, "Impact": })